In [24]:
####### imports and scripts (from http://colinraffel.com/projects/lmd/)

# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pretty_midi
import librosa
import mir_eval
# import mir_eval.display
#import tables
import IPython.display
import os
import json
import csv
import sys

# Local path constants
DATA = 'data'
RESULTS_PATH = '/Users/joro/workspace/lakh_vocal_segments_dataset/'
# Path to the file match_scores.json distributed with the LMD
SCORE_FILE = os.path.join(RESULTS_PATH, 'match_scores.json')

# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

def msd_id_to_mp3(msd_id):
    """Given an MSD ID, return the path to the corresponding mp3"""
    return os.path.join(DATA, 'msd', 'mp3',
                        msd_id_to_dirs(msd_id) + '.mp3')

def msd_id_to_h5(h5):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(RESULTS_PATH, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

def get_midi_path(msd_id, midi_md5, kind):
    """Given an MSD ID and MIDI MD5, return path to a MIDI file.
    kind should be one of 'matched' or 'aligned'. """
    return os.path.join(RESULTS_PATH, 'lmd_{}'.format(kind),
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')

In [56]:

def get_best_MIDI(msd_id):
    '''
    grab a MIDI for a given MSD track ID.
    @author georgi, adapted from  craffel
    
    '''
    with open(SCORE_FILE) as f:
        scores = json.load(f)

        # Grab the dictionary of MIDI matches for a trackID 
    if msd_id not in scores:
        sys.exit('{} is not matched'.format(msd_id))
    matches = scores[msd_id]

#     ######### get MIDI from the matches. 
#     # grab a random one
#     midi_md5, score = matches.popitem()
#     print '  {} with confidence score {}'.format(midi_md5, score)

    # or check score confidence values
    max_score_aligned_MIDI = -1
    midi_md5_max = -1
    for midi_md5, score in scores[msd_id].items():
        print '  {} with confidence score {}'.format(midi_md5, score)
        if score > max_score_aligned_MIDI:
            midi_md5_max = midi_md5
    if midi_md5_max == -1:
        sys.exit('no MIDI match found for midi {}'.format(midi_md5))

        # Construct the path to the aligned MIDI
    aligned_midi_path = get_midi_path(msd_id, midi_md5_max, 'aligned')
        # Load/parse the MIDI file with pretty_midi
    pm = pretty_midi.PrettyMIDI(aligned_midi_path)
    return pm

In [156]:


def derive_note_annotation(midi, WHICH_CHANNEL_VOCAL):
    '''
    midi:  object of pretty MIDI library 
    WHICH_CHANNEL_VOCAL: channel of vocal, has to be checked for each track separately
    
    from http://nbviewer.jupyter.org/github/craffel/midi-dataset/blob/master/Tutorial.ipynb
    '''

    vocal_channel = midi.instruments[WHICH_CHANNEL_VOCAL]
    intervals = np.array([[note.start, note.end] for note in vocal_channel.notes]) # note intervals
    notes = np.array([note.pitch for note in vocal_channel.notes]) # MIDI numbers
    return intervals, notes 

def write_notes_to_file(intervals, notes, msd_id):
    
    URI_MSD_ID = os.path.join(RESULTS_PATH, DATA, msd_id )
    if not os.path.exists(URI_MSD_ID):
        os.makedirs(URI_MSD_ID)
    URI_notes_aligned_output = os.path.join(URI_MSD_ID, 'alignedNotes_vocal.txt') # convetion name for this repo 
    
    ### write to output file
    f = open(URI_notes_aligned_output,'wb')
    writer = csv.writer(f, delimiter='\t')

    for interval, note in zip(intervals, notes): # read intervals

        row = [interval[0]] # onset time
        row.append(interval[1]-interval[0]) # duration
        row.append(note) # MIDI number
        # row.append(0) # satisfy sonic visualiser's format

        writer.writerow(row)

    f.close()
    print 'written file ' + URI_notes_aligned_output

In [85]:

def derive_beat_annotation(midi):
    '''
    from http://nbviewer.jupyter.org/github/craffel/midi-dataset/blob/master/Tutorial.ipynb
    '''
    # Retrieve the beats and downbeats from pretty_midi
    # Note that the beat phase will be wrong until the first time signature change after 0s
    # So, let's start beat tracking from that point
    first_ts_after_0 = [ts.time for ts in midi.time_signature_changes if ts.time > 0.][0]
    # Get beats from pretty_midi, supplying a start time
    beats = midi.get_beats(start_time=first_ts_after_0)
    # .. downbeats, too
    downbeats = midi.get_downbeats(start_time=first_ts_after_0)
    return beats, downbeats


def write_beats_to_file(beats, msd_id):
    '''
    assumes in 4/4 and that first beat is downbeat (beat 1) 
    '''
    URI_MSD_ID = os.path.join(RESULTS_PATH, DATA, msd_id )
    if not os.path.exists(URI_MSD_ID):
        os.makedirs(URI_MSD_ID)
        
    URI_notes_aligned_output = os.path.join(URI_MSD_ID, msd_id + '.beats_tab') # convetion name for this repo 
    
     
    BEATS_PER_BAR = 4
    ### write to output file
    f = open(URI_notes_aligned_output,'wb')
    writer = csv.writer(f, delimiter='\t')
    
    beat_num = 0 # assume starts at 1
    for beat_ts in beats: # read intervals

        row = [beat_ts + 0.1] # beat time
        beat_num= (beat_num  + 1) % BEATS_PER_BAR
        if beat_num == 0: beat_num =+  BEATS_PER_BAR # show last beat (e.g. 4) instead of 0
        row.append(beat_num) # beat number

        writer.writerow(row)

    f.close()
    print 'written file ' + URI_notes_aligned_output

In [189]:
### 2) get matched MIDI

MSD_ID = 'TRALLSG128F425A685' # idx 4 # white flag
# MSD_ID = 'TRGWUEG128F4270721' # idx 2 # africa
# MSD_ID = 'TRAYOPU128F1464B52';# 73 #sunrise
# MSD_ID = 'TRTQHCR128F42640F5' #  81 # smells like
# MSD_ID = 'TROEEIY12903CCF25A' # madona  # 75

# MSD_ID = 'TRWTFEA128F426E1DC' # etenral # 75
# MSD_ID = 'TRTRJRT12903D05FC3' # call #11
midi = get_best_MIDI(MSD_ID)

### get estimated  tempo
import numpy as np
print np.median(midi.get_tempo_changes()[1])


  904017ae5e1ceb7ab2c17022c8db49d9 with confidence score 0.503956787309
86.1328915673


In [ ]:
### 3) derive note onset annotations
#  check for each track separately where is the vocal
WHICH_CHANNEL_VOCAL = -1
for i, instrument in enumerate(midi.instruments):
    print instrument.name
    if instrument.name == 'vocals' or instrument.name == 'MELODY' or instrument.program == 75:
        WHICH_CHANNEL_VOCAL = i; break

# print midi.instruments
# WHICH_CHANNEL_VOCAL = 12
intervals, notes  = derive_note_annotation(midi, WHICH_CHANNEL_VOCAL)
write_notes_to_file(intervals, notes, MSD_ID)

In [176]:
### 4) derive beat annotations
beats, downbeats = derive_beat_annotation(midi)
write_beats_to_file (beats, MSD_ID)

written file /Users/joro/workspace/lakh_vocal_segments_dataset/data/TRTRJRT12903D05FC3/alignedNotes_vocal.txt
written file /Users/joro/workspace/lakh_vocal_segments_dataset/data/TRTRJRT12903D05FC3/TRTRJRT12903D05FC3.beats_tab
